# RAG融合

从[这个GitHub仓库](https://github.com/Raudaschl/rag-fusion)重新实现，所有的功劳归原作者所有

> RAG-Fusion是一种搜索方法，旨在弥合传统搜索范式与人类查询的多方面维度之间的差距。受到检索增强生成（RAG）的能力的启发，该项目通过使用多个查询生成和互惠排名融合来重新排列搜索结果。

## 设置

在本示例中，我们将使用Pinecone和一些虚假数据。要配置Pinecone，请设置以下环境变量：

- `PINECONE_API_KEY`：您的Pinecone API密钥

In [ ]:
# 导入OpenAIEmbeddings类和PineconeVectorStore类
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

In [ ]:
# 创建一个包含文档名称和对应主题的字典
all_documents = {
    "doc1": "Climate change and economic impact.",
    "doc2": "Public health concerns due to climate change.",
    "doc3": "Climate change: A social perspective.",
    "doc4": "Technological solutions to climate change.",
    "doc5": "Policy changes needed to combat climate change.",
    "doc6": "Climate change and its impact on biodiversity.",
    "doc7": "Climate change: The science and models.",
    "doc8": "Global warming: A subset of climate change.",
    "doc9": "How climate change affects daily weather.",
    "doc10": "The history of climate change activism.",
}

In [ ]:


# 创建一个 PineconeVectorStore 对象，并使用 PineconeVectorStore.from_texts() 方法从文本中创建向量存储
# 参数说明：
# - list(all_documents.values())：将所有文档的值转换为列表，作为输入文本
# - OpenAIEmbeddings()：使用 OpenAIEmbeddings 模型进行嵌入
# - index_name="rag-fusion"：设置索引名称为 "rag-fusion"
vectorstore = PineconeVectorStore.from_texts(
    list(all_documents.values()), OpenAIEmbeddings(), index_name="rag-fusion"
)

## 定义查询生成器

我们现在将定义一个链来进行查询生成。

In [7]:
# 导入输出解析器模块中的StrOutputParser类
from langchain_core.output_parsers import StrOutputParser
# 导入langchain_openai模块中的ChatOpenAI类
from langchain_openai import ChatOpenAI

In [68]:
# 导入langchain库中的hub模块
from langchain import hub

# 从langchain-ai/rag-fusion-query-generation模型中拉取prompt
prompt = hub.pull("langchain-ai/rag-fusion-query-generation")

In [3]:
# 创建一个聊天提示模板，模板中包含了多个对话消息
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that generates multiple search queries based on a single input query."),  # 系统消息，提示助手的角色和功能
    ("user", "Generate multiple search queries related to: {original_query}"),  # 用户消息，输入一个查询关键词
    ("user", "OUTPUT (4 queries):")  # 用户消息，输出查询结果的标题
])

In [5]:
generate_queries = (
    prompt | ChatOpenAI(temperature=0) | StrOutputParser() | (lambda x: x.split("\n"))
)

## 定义完整的链路

现在我们可以将所有内容放在一起并定义完整的链路。这个链路包括：

    1. 生成一系列查询
    2. 在检索器中查找每个查询
    3. 使用倒数排名融合将所有结果合并在一起
    
    
请注意，它不会执行最后的生成步骤。

In [50]:
原始查询 = "气候变化的影响"

In [75]:
# 从现有索引中创建PineconeVectorStore对象
vectorstore = PineconeVectorStore.from_existing_index("rag-fusion", OpenAIEmbeddings())

# 将PineconeVectorStore对象转换为检索器对象
retriever = vectorstore.as_retriever()

In [76]:
from langchain.load import dumps, loads



def reciprocal_rank_fusion(results: list[list], k=60):
    # 创建一个空字典用于存储融合后的分数
    fused_scores = {}
    for docs in results:
        # 假设文档按相关性排序返回
        for rank, doc in enumerate(docs):
            # 将文档转换为字符串
            doc_str = dumps(doc)
            # 如果文档不在融合分数字典中，则将其添加，并初始化分数为0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # 获取之前的分数
            previous_score = fused_scores[doc_str]
            # 更新融合分数，使用倒数排名加权
            fused_scores[doc_str] += 1 / (rank + k)

    # 对融合分数进行排序，并将文档和分数转换为原始格式
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [77]:
# 定义一个数据处理流程，包括生成查询、检索和倒数排名融合
chain = generate_queries | retriever.map() | reciprocal_rank_fusion

In [78]:
# 调用chain对象的invoke方法，传入一个包含原始查询的字典作为参数
chain.invoke({"original_query": original_query})

[(Document(page_content='Climate change and economic impact.'),
  0.06558258417063283),
 (Document(page_content='Climate change: A social perspective.'),
  0.06400409626216078),
 (Document(page_content='How climate change affects daily weather.'),
  0.04787506400409626),
 (Document(page_content='Climate change and its impact on biodiversity.'),
  0.03306010928961749),
 (Document(page_content='Public health concerns due to climate change.'),
  0.016666666666666666),
 (Document(page_content='Technological solutions to climate change.'),
  0.016666666666666666),
 (Document(page_content='Policy changes needed to combat climate change.'),
  0.01639344262295082)]